In [ ]:
#インポート
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import openpyxl
import git
import os
import pickle

#ローカルファイルの読み込み
import data_make as dm   #データを取得するためのファイル
import Input_form as Info #ファイル名の入力などを行うためのファイル

In [ ]:
with open('java_line.bin', 'rb') as f:
    java_line = pickle.load(f) # load commit_set
with open('java_commit.bin', 'rb') as f:
    java_commit = pickle.load(f)
with open('prob_list.bin', 'rb') as f:
    prob_list = pickle.load(f)
with open('java_module.bin', 'rb') as f:
    java_module = pickle.load(f)

In [ ]:
#結果の出力
print('java_module')
print('依存関係の数: '+ str(len(java_module)))

print('java_line')
print('ファイル数: '+ str(len(java_line)))
print('java_commit')
print('コミットファイル数: ',str(len(java_commit)))
print('prob_list')
print(len(prob_list))

In [ ]:
#空間スキャン検定に必要なファイルの読み込み
import CircularScan as cs

In [ ]:
scan_method, scan_type, graph_type, threshold, iter_num, hop_count = Info.data_get2()
threshold = float(threshold)
iter_num = int(iter_num)
hop_count = int(hop_count)

In [ ]:
int(len(java_module)/2)

In [ ]:
#依存関係の選択
if graph_type == '有効グラフ' and exception_select == "あり":
    new_java_module = java_module[0:int(len(java_module)/2)]  #<-- 有向グラフ 例外処理あり
elif graph_type == '無向グラフ' and exception_select == "あり":
    new_java_module = java_module  #<-- 無向グラフ 例外処理あり
else:
    print('該当する依存関係なし')

#実行
if scan_method == 'repeat':
    result = cs.CS_run_repeat(java_line, java_commit, new_java_module, prob_list, scan_type, iter_num, hop_count, threshold)
elif scan_method == 'single':
    result = cs.CS_run_single(java_line, java_commit, new_java_module, prob_list, scan_type, iter_num, hop_count, threshold)

In [ ]:
result

In [ ]:
#結果の保存
result_path = '../result'

if not os.path.exists(result_path):
    os.mkdir(result_path)
    print(result_path + 'を作成しました.')

#保存ファイル名の決定
Excel_name = 'result_JSP_0.9.9-0.9.10_from_ver_dependency'

#データフレームへの変換
df = pd.DataFrame(result, columns=['ファイル名','p値','LLR値'])

#エクセルデータとして結果を保存
df.to_excel(result_path+'/'+Excel_name+'.xlsx', sheet_name='new_sheet_name')

In [ ]:
#実験のログを保存
log = ['リポジトリ : ' + repo_url + '\n',
     'ブランチ : ' + from_ver + ' - ' + to_ver + '\n',
     '結果ファイル : ' + Excel_name + '.xlsx \n',
     'CSの関数 : '       + scan_method + '\n' , 
     '停止条件の種類 : ' + scan_type + '\n' ,
     'グラフの種類 : '   + graph_type + '\n', 
     'しきい値 : '       + str(threshold) + '\n' ,
     'CSの実行回数 : '   + str(iter_num) + '\n',
     'CSのホップ数 : '   + str(hop_count)]

#書き込み
with open(result_path + '/' + Excel_name + '_result_log.txt', mode='w') as f:
    f.writelines(log)

with open(result_path + '/' + Excel_name + '_result_log.txt') as f:
    print(f.read())